

Note:
    
    
    Create a folder and put the .ipynb file, train.pkl, test.pkl, and sample_submission.csv in that folder
    
    
    This folder is considered the root of the project
    
    
    Do not run the cells before doing the above mentiond instructions

In [ ]:
%%capture
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import WeightedRandomSampler
import matplotlib.pyplot as plt
import time
import os
import copy
import shutil
import pandas as pd
import random
import os
import pickle
import transformers
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import AdamW
import heapq
from scipy.stats import spearmanr
import itertools

# Preparing Data

In [ ]:
def download_data():
    
    %cd $root
    if os.path.isdir(path_data) == False:
        os.makedirs(path_data)
    %cd $path_data

    # Download your kaggle coockies file from your kaggle account and put it in root directory
    !cp ../kaggle.json ~/.kaggle/
    !kaggle competitions download -c stat946winter2021dc2
    !unzip stat946winter2021dc2.zip
    
def create_data_path():
    
    # Assumes test.pk and train.pkl are in the root directory
    
    %cd $root
    if os.path.isdir(path_data) == False:
        os.makedirs(path_data)

    !mv test.pkl $path_data
    !mv train.pkl $path_data
    !mv sample_submission.csv $path_data
    

def pkl_to_dict(data='train'):

    %cd $path_data

    if data == 'train':
        FILE_NAME = 'train.pkl'
        infile = open(FILE_NAME,'rb')
        data_set = pickle.load(infile)

    if data == 'test':
        FILE_NAME = 'test.pkl'
        infile = open(FILE_NAME,'rb')
        data_set = pickle.load(infile)

    return data_set



def load_text(train_set, samples_per_parag=10):

  # Train sentences and labels

    train_texts = {'first_sent': [], 'second_sent': [], "labels": []}

  # Creating all correct sequences in a paragraph
    correct_seq = []
    for l in range(5):
        for k in range(l, 5):
            for r in range(k + 1, 6):
                correct_seq.append((l,k,k+1,r))

  
    for parag in train_set:
    
        # Selecting samples_per_parag samples from all possible correct samples
        correct_sample_indx = random.sample(range(0, len(correct_seq)), samples_per_parag)

        # Adding samples_per_parag correct pair sentences obtained from each paragraph
        for i in range(len(correct_sample_indx)):
            corr_permut = correct_seq[correct_sample_indx[i]]
            first_sent = returning_sent_by_pos(parag['sentences'], parag["indexes"], corr_permut[0], corr_permut[1])
            second_sent = returning_sent_by_pos(parag['sentences'], parag["indexes"], corr_permut[2], corr_permut[3])

      
            train_texts['first_sent'].append(first_sent)
            train_texts['second_sent'].append(second_sent)
            train_texts['labels'].append(1)
    
    
    
        correct_sample_indx = random.sample(range(0, len(correct_seq)), samples_per_parag)

        # Adding samples_per_parag incorrect piar sentences (with label 0)
        for i in range(len(correct_sample_indx)):
            corr_permut = correct_seq[correct_sample_indx[i]]

            # Obtaining an incorrect permutation from a correct one
            incorr_permut = (corr_permut[2], corr_permut[3], corr_permut[0], corr_permut[1])
            first_sent = returning_sent_by_pos(parag['sentences'], parag["indexes"], incorr_permut[0], incorr_permut[1])
            second_sent = returning_sent_by_pos(parag['sentences'], parag["indexes"], incorr_permut[2], incorr_permut[3])

            train_texts['first_sent'].append(first_sent)
            train_texts['second_sent'].append(second_sent)
            train_texts['labels'].append(0)

  
    return train_texts


def load_text_2seq_all_pert(train_set, samples_per_parag):
    train_texts = {'first_sent': [], 'second_sent': [], "labels": []}
    correct_seq = [(0,1), (1,2), (2,3), (3,4), (4,5)]
    
    all_pairs = []
    for i in range(6):
        for j in range(i+1, 6):
            if j == i + 1:
                for k in range(5):
                    all_pairs.append((i, j))
                all_pairs.append((j,i))
            else:
                all_pairs.append((i,j))
                all_pairs.append((j,i))
                
    for parag in train_set:

        sample_indx = random.sample(range(0, len(all_pairs)), samples_per_parag)

        for i in range(len(sample_indx)):
            permut = all_pairs[sample_indx[i]]
            first_sent = returning_sent_by_pos(parag['sentences'], parag["indexes"], permut[0], permut[0])
            second_sent = returning_sent_by_pos(parag['sentences'], parag["indexes"], permut[1], permut[1])


            train_texts['first_sent'].append(first_sent)
            train_texts['second_sent'].append(second_sent)
            
            if permut[0] + 1 == permut[1]:
                train_texts['labels'].append(1)

            else:
                train_texts['labels'].append(0)
        
    return train_texts


def load_text_2seq(train_set, samples_per_parag=5):

  # Train sentences and labels

    train_texts = {'first_sent': [], 'second_sent': [], "labels": []}

  # Creating all correct sequences in a paragraph
    correct_seq = [(0,1), (1,2), (2,3), (3,4), (4,5)]
    
    for parag in train_set:
    
        # Selecting samples_per_parag samples from all possible correct samples
        correct_sample_indx = random.sample(range(0, len(correct_seq)), samples_per_parag)

        # Adding samples_per_parag correct pair sentences obtained from each paragraph
        for i in range(len(correct_sample_indx)):
            corr_permut = correct_seq[correct_sample_indx[i]]
            first_sent = returning_sent_by_pos(parag['sentences'], parag["indexes"], corr_permut[0], corr_permut[0])
            second_sent = returning_sent_by_pos(parag['sentences'], parag["indexes"], corr_permut[1], corr_permut[1])

      
            train_texts['first_sent'].append(first_sent)
            train_texts['second_sent'].append(second_sent)
            train_texts['labels'].append(1)
    
    
        
        condidates = []
        
        # Adding samples_per_parag incorrect piar sentences (with label 1)
        while len(condidates) < samples_per_parag:
            indices = [1,2,3,4,5]
            
            f_index = random.sample(indices,1) # Selecting the first sentence
            
            for j in range(f_index[0],6):
                indices.remove(j)
            indices.append(0)
            
            s_index = random.sample(indices,1) # The second sentence should not be after the first one
            if (f_index[0], s_index[0]) not in condidates:
                condidates.append((f_index[0], s_index[0]))
            
            first_sent = returning_sent_by_pos(parag['sentences'], parag["indexes"], f_index[0], f_index[0])
            second_sent = returning_sent_by_pos(parag['sentences'], parag["indexes"], s_index[0], s_index[0])

            train_texts['first_sent'].append(first_sent)
            train_texts['second_sent'].append(second_sent)
            train_texts['labels'].append(0)


  
    return train_texts



def returning_sent_by_pos(parag_list, indices, i , j):
  # we assume that j is bigger than or equal to i

    final_sent = ""
    for sent in range(i, j+1):
        final_sent += parag_list[indices.index(sent)] + ' '
  
    return final_sent


In [ ]:
# ============== Run this cell only once ========== Run this cell only once ============ Run this cell only once

root = %pwd 
path_data = data_root = os.path.join(root, "data/")
create_data_path()

In [ ]:
# Reading the data from pkl file
train_set = pkl_to_dict('train')
test_set = pkl_to_dict('test')

# redcuing the volume of data
# Get list of paragraphs in form of a single string
# train_texts = load_text_2seq(train_set[:10],4)
# train_texts = load_text(train_set[:20000],8)
train_texts = load_text_2seq_all_pert(train_set[:200000],15)
del train_set

# Seperating 10% of train data for validation
val_texts = {'first_sent': [], 'second_sent': [], "labels": []}
train_texts['first_sent'], val_texts['first_sent'], train_texts['second_sent'], val_texts['second_sent'], train_texts['labels'], val_texts['labels'],  = train_test_split(train_texts['first_sent'], train_texts['second_sent'], train_texts['labels'], test_size=.1)


# Tokenizing

In [ ]:
def save_obj(obj, path='train'):
  
    %cd $path_data

    with open(path + ".txt", "wb") as fp:
        pickle.dump(obj, fp)




def load_saved_obj(path='train'):

    %cd $path_data

    with open(path + ".txt", "rb") as fp:
        obj = pickle.load(fp)


    return obj

In [ ]:
tokenizer = transformers.DebertaTokenizer.from_pretrained('microsoft/deberta-base') 

In [ ]:
max_length = 150
train_encodings = tokenizer(text=train_texts['first_sent'], text_pair=train_texts['second_sent'], add_special_tokens=True, truncation=True, padding=True, return_token_type_ids=True, max_length=max_length)
val_encodings = tokenizer(text=val_texts['first_sent'], text_pair=val_texts['second_sent'], add_special_tokens=True, truncation=True, padding=True, return_token_type_ids=True, max_length=max_length)
train_encodings['labels'] = train_texts['labels']
val_encodings['labels'] = val_texts['labels']

print("saving")
# save_obj(train_encodings, path='train_encodings_120-220k_AllPerBin_DeBerta_150mxl_#Sam15')
# save_obj(val_encodings, path='val_encodings_120-220k_AllPerBin_DeBerta_150mxl_#Sam15')

print('loading saved tokenized train encoding dictionary')
# train_encodings = load_saved_obj(path='train_encodings_120k_AllPerBin_DeBerta_150mxl_#Sam10')
# val_encodings = load_saved_obj(path='val_encodings_120k_AllPerBin_DeBerta_150mxl_#Sam10')


# Creating Dataset

In [ ]:
class NSPDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
#         self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         if self.labels != None:
#           item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        # return len(self.labels)
        return len(self.encodings['input_ids'])

In [ ]:
train_dataset = NSPDataset(train_encodings)
val_dataset = NSPDataset(val_encodings)

# Creating DeBerta Model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


batch_size = 16

model = transformers.DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels = 2)

model.to(device)
model.train()

In [ ]:

# Cheking the model

model.to(device)
model.train()
s_sent = [' This is a very good completely a' , 'akbar please how are different', 'akbar please how are different', ' This is a very good completely a' , 'akbar please how are different', 'akbar please how are different']
f_sent = ['hello my friend', 'She you you me me you', 'She you you me me you', 'hello my friend', 'She you you me me you', 'She you you me me you']
toks = tokenizer(text=f_sent, text_pair=s_sent, add_special_tokens=True, truncation=True, padding=True, return_token_type_ids=True)
toks['labels'] = [1,1,1,1,1,1]
toks = NSPDataset(toks)
tok_loader = DataLoader(toks, batch_size=6, shuffle=False)
for toki in tok_loader:
    input_ids = toki['input_ids'].to(device)
    attention_mask = toki['attention_mask'].to(device)
    labels = toki['labels'].to(device)
    token_type_ids=toki['token_type_ids'].to(device)
    output = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    print(output.logits.shape)

In [ ]:

def save_model(model, acc, path='NSPDeberta'):
  
    %cd $root

    model_path = os.path.join(root, 'BestModel/')
    if os.path.isdir(model_path) == False:
        os.makedirs(model_path)
    
    %cd $model_path

    torch.save(model.state_dict(), path + 'acc_' + str(acc))

    
def load_model(model, name):
    
    model_path = os.path.join(root, 'BestModel/')
    %cd $model_path
    model.load_state_dict(torch.load(name))
    return model  

# Training

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=70,
    learning_rate=8e-6,
    save_steps=8000,
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

# Creating the correct permutation using different algorithms

In [ ]:
def score(f_seq, s_seq, test_parag, model, tokenizer):
    f_sent = ''
    s_sent = ''
    for car in f_seq:
        f_sent += test_parag['sentences'][int(car)] + ' '
    
    for car in s_seq:
        s_sent += test_parag['sentences'][int(car)] + ' '
        
    toks = tokenizer(text=f_sent, text_pair=s_sent, add_special_tokens=True, truncation=True, padding=True, return_token_type_ids=True, return_tensors='pt')
    
    input_ids = toks['input_ids'].to(device)
    attention_mask = toks['attention_mask'].to(device)
    token_type_ids=toks['token_type_ids'].to(device)
    
    output = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    
    return -1 * output.logits.squeeze().softmax(dim=0)[1].item()
    

In [ ]:
def search_seq(test_parag, model, tokenizer, beam_size=3):
    

    pq = [] # Priority Queue
    indx = ['0','1','2','3','4','5']
    used = {}
    for i in range(len(indx)):
        for j in range(i+1, len(indx)):
            heapq.heappush(pq, (score(indx[i], indx[j], test_parag, model, tokenizer), indx[i] + indx[j]))
            heapq.heappush(pq, (score(indx[j], indx[i], test_parag, model, tokenizer), indx[j] + indx[i]))

            used.update({indx[i] + indx[j]: score(indx[i],indx[j],test_parag, model, tokenizer)})
            used.update({indx[j] + indx[i]: score(indx[j],indx[i],test_parag, model, tokenizer)})


    bool_break = False
    final_seq = ''
    i = 0

    while True:

    #     best_seq = heapq.nsmallest(beam_size,pq, key=lambda s: s[0])

        best_seq = []
        for beam in range(beam_size):
            pair = heapq.heappop(pq)
            best_seq.append(pair)
            sequence = pair[1]
            if len(sequence) == 6:
                final_seq = pair
                bool_break = True



        if bool_break:
            break


        for pair in best_seq:
            sent_str = pair[1]
            indx = ['0','1','2','3','4','5']

            # Remove existing sentences in the current sequence in order to not be included in the future sequences
            for car in sent_str:
                indx.remove(car)

            for new_sent in indx:
                cand1 = sent_str + new_sent
                cand2 = new_sent + sent_str

                if cand1 not in used:
                    score_cand1 = score(sent_str, new_sent, test_parag, model, tokenizer)
                    used.update({cand1: score_cand1})
                    heapq.heappush(pq, (score_cand1, cand1))

                if cand2 not in used:
                    score_cand2 = score(new_sent, sent_str, test_parag, model, tokenizer)
                    used.update({cand2: score_cand2})
                    heapq.heappush(pq, (score_cand2, cand2))


    return final_seq

In [ ]:
def seq_to_out(seq):
    out = []
    if type(seq) == str:
        seq = [int(i) for i in seq]
    for i in range(6):
        out.append(seq.index(i))
    return out

In [ ]:
def best_permut(test_parag, model, tokenizer):

    correct_seq = []
    indxs = ['0', '1', '2', '3', '4', '5']
    for i in range(len(indxs)):
        for j in range(i+1, len(indxs)):
            correct_seq.append((indxs[i], indxs[j]))
            correct_seq.append((indxs[j], indxs[i]))

    seq_scores = {}
    for seq in correct_seq:
        str_seq = seq[0] + seq[1]
        score_seq = score(seq[0], seq[1], test_parag, model, tokenizer)
        seq_scores.update({str_seq: score_seq})
    
    positions = ['0', '1', '2', '3', '4', '5']
    permuts = ["".join(list(i)) for i in list(itertools.permutations(positions))] # all 720 permuatations
    
    
    pq = []
    for per in permuts:
        per_score = 0
        for i in range(5):
            str_seq = per[i] + per[i+1]
            per_score += seq_scores[str_seq]
        heapq.heappush(pq, (per_score, per))
    
    return heapq.heappop(pq)
            

In [ ]:
def correct_order(model, tokenizer, test_set, b_per=True, s_seq=False, with_label=True, beam_size=3):
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
   
    resutls = {'score_s':[], 'score_b':[]}
    
    for data in tqdm(range(len(test_set))):
        
        if s_seq:
            s_raw_score, s_predicted_seq = search_seq(test_set[data], model, tokenizer, beam_size)
            s_out = seq_to_out(s_predicted_seq)
        
        if b_per:
            b_raw_score, b_predicted_seq = best_permut(test_set[data], model, tokenizer)
            b_out = seq_to_out(b_predicted_seq)
            
        
        if with_label:
            corr_seq = test_set[data]['indexes']
            if s_seq:
                resutls['score_s'].append((spearmanr(s_out, corr_seq)[0], s_predicted_seq, s_raw_score))
            
            if b_per:
                resutls['score_b'].append((spearmanr(b_out, corr_seq)[0], b_predicted_seq, b_raw_score))

        else:
            if s_seq:
                resutls['score_s'].append((s_out, s_predicted_seq, s_raw_score))
            
            if b_per:
                resutls['score_b'].append((b_out, b_predicted_seq, b_raw_score))
        
            

    return resutls


In [ ]:

# Predicting the best permutation for the test data using correct_order function.

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

test_res = correct_order(model, tokenizer, test_set, with_label=False)

In [ ]:
predictions = []
for i in range(len(test_res['score_b'])):
    predictions.append(test_res['score_b'][i][0])
predictions

In [ ]:
%cd $path_data
test_df = pd.read_csv('sample_submission.csv')
test_df.head()

In [ ]:
for i in range(0,len(predictions)):
    test_df.loc[test_df['ID'] == i+1] = [[i+1] + predictions[i]]
test_df.head()

In [ ]:
# Saving the new CSV file
%cd $path_data
test_df.to_csv('precidted.csv', index=False)

In [ ]:
pred_df = pd.read_csv('precidted.csv')
pred_df.head()

In [ ]:
# !kaggle competitions submit -c stat946winter2021dc2 -f precidted_lables.csv -m "DeBerta-0.7939-BestPer"

# =================================================